# Demonstrate How to Combine Multiple Tanks into a Network

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from cas_models.discrete_time.models import StateSpaceModelDT
from cas_models.transformations import connect_systems
from cas_models.discrete_time.simulate import make_n_step_simulation_function_from_model
from feed_conc_ctrl import MixingTankModelDT

In [ ]:
plot_dir = Path("plots")
plot_dir.mkdir(exist_ok=True)

In [ ]:
D = 5  # tank diameter [m]
n_tanks = 4
tank_names = [f"tank_{i+1}" for i in range(n_tanks)]

systems = [MixingTankModelDT(D=D, name=name) for name in tank_names]

def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

for sys in systems:
    print_sys_dimensions(sys)
    

In [ ]:
connections = {
    'tank_2_conc_in': 'tank_1_conc_out',
    'tank_3_conc_in': 'tank_1_conc_out',
    'tank_4_conc_in': ['tank_2_conc_out', 'tank_3_conc_out'],  # wrong!
    'tank_1_v_dot_out': ['tank_2_v_dot_in', 'tank_3_v_dot_in'],
    'tank_4_v_dot_in': ['tank_2_v_dot_out', 'tank_3_v_dot_out'],
}
# TODO: Need a flow-mixing system to properly combine flows from Tank 2 and 3
model_class = StateSpaceModelDT
feed_tanks_system = connect_systems(systems, connections, model_class, name="tank_system_121") 
print_sys_dimensions(feed_tanks_system)

In [ ]:
nT = 100
simulate = make_n_step_simulation_function_from_model(feed_tanks_system, nT)
simulate

## Demo simulation

In [ ]:
# Sample period
Ts = 1  # hours

# Evaluation times
t = Ts * np.arange(nT+1)

# Input signals
t_in = t[:-1]
U = np.zeros((nT, feed_tanks_system.nu))
U[t_in > 5, 0] = 10.0  # tank_1_v_dot_in
U[:, 1] = 0.5  # tank_1_conc_in
U[t_in > 10, 2] = 5.0  # tank_2_v_dot_in
U[t_in > 30, 3] = 5.0  # tank_2_v_dot_out
U[t_in > 20, 4] = 5.0  # tank_3_v_dot_in
U[t_in > 40, 5] = 5.0  # tank_3_v_dot_out
U[t_in > 50, 6] = 10.0  # tank_4_v_dot_out

# Initial conditions
x0 = [
    0.5,  # tank_1_L
    2,  # tank_1_m
    0.5,  # tank_2_L
    2,  # tank_2_m
    0.5,  # tank_3_L
    2,  # tank_3_m
    0.5,  # tank_4_L
    2,  # tank_4_m
]

X, Y = simulate(t, U, x0)

assert X.shape == (nT+1, feed_tanks_system.n)
assert Y.shape == (nT+1, feed_tanks_system.ny)

sim_results = {
    "inputs": pd.DataFrame(U, columns=feed_tanks_system.input_names),
    "states": pd.DataFrame(X, columns=feed_tanks_system.state_names),
    "outputs": pd.DataFrame(Y, columns=feed_tanks_system.output_names)
}
sim_results = pd.concat(sim_results, axis=1)
sim_results.head()


In [ ]:
flow_units = r'$m^3/h$'
conc_units = r'$tons/m^3$'
level_units = r'$m$'

units = {
    'tank_1_v_dot_in': flow_units, 
    'tank_1_v_dot_out': flow_units, 
    'tank_2_v_dot_in': flow_units, 
    'tank_2_v_dot_out': flow_units, 
    'tank_3_v_dot_in': flow_units,  
    'tank_3_v_dot_out': flow_units, 
    'tank_4_v_dot_in': flow_units, 
    'tank_4_v_dot_out': flow_units,
    'tank_1_conc_in': conc_units,
    'tank_1_conc_out': conc_units,
    'tank_2_conc_out': conc_units,
    'tank_3_conc_out': conc_units,
    'tank_4_conc_out': conc_units,
    'tank_1_L': level_units,
    'tank_2_L': level_units,
    'tank_3_L': level_units,
    'tank_4_L': level_units,
}

plot_info = {
    "Tank 1 Flows": {
        "Flow In": 'tank_1_v_dot_in',
    },
    "Tank 2 Flows": {
        "Flow In": 'tank_2_v_dot_in',
        "Flow Out": 'tank_2_v_dot_out'
    },
    "Tank 3 Flows": {
        "Flow In": 'tank_3_v_dot_in',
        "Flow Out": 'tank_3_v_dot_out'
    },
    "Tank 4 Flows": {
        "Flow Out": 'tank_4_v_dot_out'
    },
    "Tank Levels": {
        "Tank 1": 'tank_1_L',
        "Tank 2": 'tank_2_L',
        "Tank 3": 'tank_3_L',
        "Tank 4": 'tank_4_L'
    },
    "Tank Concentrations": {
        "Feed": 'tank_1_conc_in',
        "Tank 1": 'tank_1_conc_out',
        "Tank 2": 'tank_2_conc_out',
        "Tank 3": 'tank_3_conc_out',
        "Tank 4": 'tank_4_conc_out'
    },
}

In [ ]:
def make_tsplots(
    data, plot_info, units=None, time_units='hours', time_label='Time ({time_units})'
):
    n_subplots = len(plot_info)
    width, height = 8, 1 + 1.5*n_subplots

    fig, axes = plt.subplots(n_subplots, 1, sharex=True, figsize=(width, height))
    for ax, (title, sub_plot_info) in zip(axes, plot_info.items()):
        for name, var_name in sub_plot_info.items():
            data[var_name].plot(ax=ax, label=name)
        ax.set_ylabel(units[var_name])
        ax.grid(True)
        ax.legend()
        ax.set_title(title)

    axes[-1].set_xlabel(time_label.format(time_units=time_units))

    return fig, axes

var_data = sim_results.droplevel(0, axis=1)
var_data = var_data.loc[:, ~var_data.columns.duplicated()]
fig, axes = make_tsplots(var_data, plot_info, units=units)
plt.tight_layout()
plt.savefig(plot_dir / "tank_network_simulation.png")
plt.close()